In [1]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)
from keras.backend.tensorflow_backend import set_session
set_session(tf.Session(config=config))
#import keras
from keras import backend as K
K.set_image_dim_ordering('tf')
from keras.models import Model
import numpy as np
from keras.applications.imagenet_utils import preprocess_input
from keras.preprocessing import image
from matplotlib.pyplot import imshow
import pandas as pd
from keras.models import Sequential
from keras.layers import Input,Dense, Dropout, Flatten,Activation, Conv2D, MaxPooling2D,BatchNormalization,GlobalAveragePooling2D
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical

import keras
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
keras.backend.set_image_data_format("channels_last")

Using TensorFlow backend.


# Step 1:
a)creating a train generator-This train generator is attached with 120 classes each class represent a dog breed.
 this 120 floders consist of cropped images using yolo(you only look once).

b)similarly creating a validation generator with 1200 samples with 10 cropped dog samples for each class 


In [51]:
train_datagen = ImageDataGenerator(rescale=1./255,rotation_range=0.2  ,horizontal_flip=True,
    vertical_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        '/home/sainathb/Image_Classification/demo1/',
        target_size=(224,224),
        batch_size=60,class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(
        '/home/share/dog_val/',
        target_size=(224,224),
        batch_size=60,class_mode='categorical')

Found 10672 images belonging to 120 classes.
Found 1200 images belonging to 120 classes.


# Step 2:
importing Xception pre-trained model from keras with imagenet weights and freezing the convolutional layers.

In [49]:
base_model=keras.applications.xception.Xception(include_top=False, weights='imagenet',input_shape=(224,224,3),pooling='avg')
base_model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [43]:
for layer in base_model.layers:
    layer.trainable = False


In [44]:
base_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_6 (InputLayer)             (None, 224, 224, 3)   0                                            
____________________________________________________________________________________________________
block1_conv1 (Conv2D)            (None, 111, 111, 32)  864         input_6[0][0]                    
____________________________________________________________________________________________________
block1_conv1_bn (BatchNormalizat (None, 111, 111, 32)  128         block1_conv1[0][0]               
____________________________________________________________________________________________________
block1_conv1_act (Activation)    (None, 111, 111, 32)  0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

# Step 3:
Connecting Xception model to dense networks

In [45]:
model = Sequential()
model.add(base_model)
model.add(Dense(1024))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dense(120))
model.add(Activation('softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Model)             (None, 2048)              20861480  
_________________________________________________________________
dense_14 (Dense)             (None, 1024)              2098176   
_________________________________________________________________
activation_14 (Activation)   (None, 1024)              0         
_________________________________________________________________
batch_normalization_32 (Batc (None, 1024)              4096      
_________________________________________________________________
dense_15 (Dense)             (None, 120)               123000    
_________________________________________________________________
activation_15 (Activation)   (None, 120)               0         
Total params: 23,086,752
Trainable params: 2,223,224
Non-trainable params: 20,863,528
________________________________________________________

# Step 4:
Compile the model

In [52]:
model.compile(loss='categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])
model.fit_generator(train_generator, steps_per_epoch=200,
        epochs=75,validation_data=validation_generator,validation_steps=20)

Epoch 1/75
200/200 [==============================] - 180s - loss: 1.8184 - acc: 0.5598 - val_loss: 0.8055 - val_acc: 0.8117
Epoch 2/75
200/200 [==============================] - 195s - loss: 1.6305 - acc: 0.5955 - val_loss: 0.8877 - val_acc: 0.8017
Epoch 3/75
200/200 [==============================] - 221s - loss: 1.4772 - acc: 0.6247 - val_loss: 0.7711 - val_acc: 0.8158
Epoch 4/75
200/200 [==============================] - 220s - loss: 1.3921 - acc: 0.6382 - val_loss: 0.8173 - val_acc: 0.8125
Epoch 5/75
200/200 [==============================] - 221s - loss: 1.3181 - acc: 0.6552 - val_loss: 0.7566 - val_acc: 0.8233
Epoch 6/75
200/200 [==============================] - 221s - loss: 1.2507 - acc: 0.6764 - val_loss: 0.7829 - val_acc: 0.8158
Epoch 7/75
200/200 [==============================] - 220s - loss: 1.1877 - acc: 0.6904 - val_loss: 0.7643 - val_acc: 0.8242
Epoch 8/75
200/200 [==============================] - 221s - loss: 1.1343 - acc: 0.6986 - val_loss: 0.7616 - val_acc: 0.8175


200/200 [==============================] - 140s - loss: 0.2506 - acc: 0.9470 - val_loss: 0.7070 - val_acc: 0.8708
Epoch 67/75
200/200 [==============================] - 141s - loss: 0.2462 - acc: 0.9482 - val_loss: 0.7383 - val_acc: 0.8533
Epoch 68/75
200/200 [==============================] - 141s - loss: 0.2407 - acc: 0.9489 - val_loss: 0.7168 - val_acc: 0.8642
Epoch 69/75
200/200 [==============================] - 140s - loss: 0.2433 - acc: 0.9490 - val_loss: 0.7197 - val_acc: 0.8558
Epoch 70/75
200/200 [==============================] - 141s - loss: 0.2267 - acc: 0.9523 - val_loss: 0.6873 - val_acc: 0.8675
Epoch 71/75
200/200 [==============================] - 141s - loss: 0.2370 - acc: 0.9488 - val_loss: 0.7126 - val_acc: 0.8642
Epoch 72/75
200/200 [==============================] - 141s - loss: 0.2370 - acc: 0.9492 - val_loss: 0.7247 - val_acc: 0.8742
Epoch 73/75
200/200 [==============================] - 141s - loss: 0.2213 - acc: 0.9559 - val_loss: 0.7170 - val_acc: 0.8683
Epoc

# Step 5:
then extract test images and save the test file

In [ ]:
import matplotlib.pyplot as plt
import cv2
img=cv2.imread("/home/sainathb/Image_Classification/sample_submission.csv.zip.jpg")
plt.imshow(img)
plt.show()


In [ ]:
def read_img(img_path):
    img = cv2.imread(img_path)
    img = cv2.resize(img,(224,224))
    return img/255

In [ ]:
from tqdm import tqdm
import cv2
train=pd.read_csv("/home/sainathb/Image_Classification/sample_submission.csv")
trainimages=[]
for img_path in tqdm(train['id'].values):
        trainimages.append(read_img("/home/sainathb/Image_Classification/test/"+ img_path+".jpg"))

In [ ]:
trainimages=np.array(trainimages)

In [ ]:
k=model.predict(trainimages)

In [ ]:
np.savetxt('xception1.csv', k, delimiter=',') 